In [1]:
import os
execfile("../script/utils.py")
eventsPath = os.environ["YAHOO_DATA"]
splitedRdd = sc.textFile(eventsPath + "/splitedData")
splitedRdd = splitedRdd.map(parseContextData2)
splitedRdd = sc.parallelize(splitedRdd.take(2000))
#splitedRdd.take(1)
#(uid,[[train],[test]]

In [ ]:
def remove_duplicates(values):
    output = []
    seen = set()
    for value in values:
        # If value has not been encountered yet,
        # ... add it to both list and set.
        if value not in seen:
            output.append(value)
            seen.add(value)
    return output

def evaluation_rr(testList, recList):
    scores = 0
    for t in testList:
        for i in range(len(recList)):
            if t == recList[i]:
                scores = scores + 1.0 / (i+1)
                break
    scores = scores / len(testList)
    return scores

def evaluation_ap(testList, recList):
    scores = 0
    num = 0 
    rr = []
    rr[:len(recList)] = [0 for i in range(len(recList))]
    for t in testList:
        for i in range(len(recList)):
            if t == recList[i]:
                rr[i] = 1
    
    for t in testList:    
        for i in range(len(recList)):
            if t == recList[i]:
                for j in range(i,len(recList)):    
                    if rr[j] == 1:
                        scores = scores + 1.0 / (j+1)
                        num = num + 1
                break
    if num == 0:
        scores = 0
    else:
        scores = scores / num
    return scores

def evaluation_recall(testList, recList):
    scores = 0
    num = 0
    recall = []
    recall[:len(recList)] = [0 for i in range(len(recList))]
    rr=[]
    rr[:len(recList)] = [0 for i in range(len(recList))]
    for t in testList:
        for i in range(len(recList)):
            if t == recList[i]:
                rr[i] = 1
    for t in testList:    
        for i in range(len(recList)):
            if t == recList[i]:
                for j in range(i,len(recList)):    
                    recall[j] = recall[j] + 1.0
                    num = num + 1  
                break    
    recall = [0 if num == 0 else x*sum(rr)/num for x in recall]
    return recall
def print_result(finalScore, algorithm_name):
    f = open('result/resultOfEvaluation1.txt','a')
    f.write(algorithm_name + ' by mrr:'+str(finalScore.map(lambda x:x[0]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by map:'+str(finalScore.map(lambda x:x[1]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by recall:')
    for i in range(numRecList):
        if i == 0:
            f.write(' '+str(finalScore.map(lambda x:x[2][i]).mean()))
        elif i == numRecList-1:
            f.write(','+str(finalScore.map(lambda x:x[2][i]).mean()) + '\n')
        else:
            f.write(','+str(finalScore.map(lambda x:x[2][i]).mean()))
    f.close()

In [ ]:
numRecList = 5
def lruFunction(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    numTrain = len(trainList)
    numTest = l - numTrain
    trainList = sorted(trainList,key=lambda x:int(x[1][0]), reverse=True);  #sort by timestamp with descending
    testList = [t[0] for t in testList]                      #take only id for test set
    RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
    Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate
    
    if len(Recommender) > numRecList-1:
        finalRecommender = Recommender[:numRecList]
    else:    
        finalRecommender = [-1,-1,-1,-1,-1]
        numRec = len(Recommender)
        finalRecommender[:numRec] = Recommender
    
    scores_rr = evaluation_rr(testList, finalRecommender)
    scores_ap = evaluation_ap(testList, finalRecommender)
    scores_recall = evaluation_recall(testList, finalRecommender)
    return scores_rr, scores_ap, scores_recall#[0],scores_recall[1],scores_recall[2],scores_recall[3],scores_recall[4]

finalScore = splitedRdd.map(lruFunction)
#finalScore.map(lambda x:x[2][0]).mean()
#finalScore.take(10)#,finalScore.map(lambda x:x[2]).mean()
print_result(finalScore, 'lru')

In [ ]:
def mruFunction(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    numTrain = len(trainList)
    numTest = l - numTrain    
    #trainList = sorted(trainList,key=lambda x:int(x[1]), reverse=True);  #sort by timestamp with descending
    testList = [t[0] for t in testList]                      #take only id for test set
    RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
    Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate
    
    if len(Recommender) > 4:
        finalRecommender = Recommender[:5]
    else:
        finalRecommender = [-1,-1,-1,-1,-1]
        numRec = len(Recommender)
        finalRecommender[:numRec] = Recommender
    scores_rr = evaluation_rr(testList, finalRecommender)
    scores_ap = evaluation_ap(testList, finalRecommender)
    scores_recall = evaluation_recall(testList, finalRecommender)
    return scores_rr, scores_ap, scores_recall
finalScore = splitedRdd.map(mruFunction)
print_result(finalScore,'mru')

In [ ]:
from collections import Counter
def mfuFunction(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    numTrain = len(trainList)
    numTest = l - numTrain   
    
    testList = [t[0] for t in testList]                      #take only id for test set
    RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
    #Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate
    Recommender = Counter(RecommenderDuplicate).most_common()
    Recommender =  [t[0] for t in Recommender] 

    if len(Recommender) > 4:
        finalRecommender = Recommender[:5]
    else:
        finalRecommender = [-1,-1,-1,-1,-1]
        numRec = len(Recommender)
        finalRecommender[:numRec] = Recommender
    scores_rr = evaluation_rr(testList, finalRecommender)
    scores_ap = evaluation_ap(testList, finalRecommender)
    scores_recall = evaluation_recall(testList, finalRecommender)
    return scores_rr, scores_ap, scores_recall
finalScore = splitedRdd.map(mfuFunction)
print_result(finalScore,'mfu')